In [1]:
!ls ../mrt_nation_list.csv

../mrt_nation_list.csv


# 1. 국가 리스트 데이터 불러오기

In [2]:
nation_df = pd.read_csv("../mrt_nation_list.csv")

In [3]:
del nation_df["Unnamed: 0"]

In [4]:
nation_df

,nations,nations_url,continent,continental_region,nations_eng
0,네덜란드,/countries?key_name=Netherlands,유럽,서유럽,Netherlands
1,독일,/countries?key_name=Germany,유럽,서유럽,Germany
2,룩셈부르크,/countries?key_name=Luxembourg,유럽,서유럽,Luxembourg
3,모나코,/countries?key_name=Monaco,유럽,서유럽,Monaco
4,벨기에,/countries?key_name=Belgium,유럽,서유럽,Belgium
...,...,...,...,...,...
102,알제리,/countries?key_name=Algeria,아프리카,남태평양,Algeria
103,이집트,/countries?key_name=Egypt,아프리카,남태평양,Egypt
104,잠비아,/countries?key_name=Zambia,아프리카,남태평양,Zambia
105,케냐,/countries?key_name=Kenya,아프리카,남태평양,Kenya


In [5]:
nation_df["continent"].value_counts()

유럽       41
아시아      30
아메리카     20
아프리카     10
오세아니아     6
Name: continent, dtype: int64

In [6]:
nation_df["continental_region"].value_counts()

중남미                    18
동유럽 / 발틱3국             14
남태평양                   13
남유럽 / 지중해 / 터키         11
서유럽                    11
동남아                    10
인도 / 중남아시아              7
중동                      6
북유럽                     5
홍콩 / 마카오 / 대만 / 싱가폴     4
호주 / 뉴질랜드               3
중국 / 일본                 2
미국 / 캐나다                2
국내                      1
Name: continental_region, dtype: int64

# 2. 도시 리스트 데이터 불러오기

In [7]:
city_df = pd.read_csv("../mrt_city_list.csv")

In [8]:
del city_df["Unnamed: 0"]

In [9]:
city_df

,nation_kor,nation_eng,city_name_eng,city_name_kor
0,네덜란드,Netherlands,Amsterdam,암스테르담
1,네덜란드,Netherlands,Rotterdam,로테르담
2,네덜란드,Netherlands,Hague,헤이그
3,네덜란드,Netherlands,Giethoorn,히트호른
4,네덜란드,Netherlands,Volendam,볼렌담
...,...,...,...,...
777,이집트,Egypt,Luxor,룩소르
778,잠비아,Zambia,Livingstone,리빙스턴
779,케냐,Kenya,Nairobi,나이로비
780,탄자니아,"Tanzania,+United+Republic+of",Moshi,모시


In [10]:
city_count = city_df["nation_kor"].value_counts()

In [11]:
city_count_df = pd.DataFrame(city_count)
city_count_df.loc[city_count_df["nation_kor"] >= 10, :]

,nation_kor
일본,62
대한민국,58
미국,49
독일,42
이탈리아,36
프랑스,33
스페인,28
중국,25
영국,25
인도,22


In [12]:
nation_df

,nations,nations_url,continent,continental_region,nations_eng
0,네덜란드,/countries?key_name=Netherlands,유럽,서유럽,Netherlands
1,독일,/countries?key_name=Germany,유럽,서유럽,Germany
2,룩셈부르크,/countries?key_name=Luxembourg,유럽,서유럽,Luxembourg
3,모나코,/countries?key_name=Monaco,유럽,서유럽,Monaco
4,벨기에,/countries?key_name=Belgium,유럽,서유럽,Belgium
...,...,...,...,...,...
102,알제리,/countries?key_name=Algeria,아프리카,남태평양,Algeria
103,이집트,/countries?key_name=Egypt,아프리카,남태평양,Egypt
104,잠비아,/countries?key_name=Zambia,아프리카,남태평양,Zambia
105,케냐,/countries?key_name=Kenya,아프리카,남태평양,Kenya


In [13]:
nations = nation_df["nations_eng"]
nations[:5]

0    Netherlands
1        Germany
2     Luxembourg
3         Monaco
4        Belgium
Name: nations_eng, dtype: object

# 3. 국가명에 따른 상품 코드와 기본 정보 가져오는 함수
### 함수 실행 순서 : get_url -> get_offer_info -> get_result_pd 순서로 실행된다.

In [14]:
import pandas as pd
import requests
import re

from scrapy.http import TextResponse

In [15]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

### 전체 국가 107개

In [124]:
nation_names = nations[0:107]

In [125]:
nation_names

0                         Netherlands
1                             Germany
2                          Luxembourg
3                              Monaco
4                             Belgium
                    ...              
102                           Algeria
103                             Egypt
104                            Zambia
105                             Kenya
106    Tanzania%2C+United+Republic+of
Name: nations_eng, Length: 107, dtype: object

### url 가져오는 함수

In [170]:
def get_url(name):
    
    url_base = "https://www.myrealtrip.com"
    
    urls = []
    for sel_nation in name:
        url_sub1 = "/offers?country={}".format(sel_nation)

        for page in range(1): ### 첫번째 페이지만 가져오기
            url_sub2 = url_sub1 + "&group_category=experience&order=reviews&page={}".format(page) ### order=reviews는 후기순
            url_sum = url_base + url_sub2
            urls.append(url_sum)
            
    return get_offer_info(urls) ### 첫번째 페이지의 html 정보를 가져오는 함수를 return

https://www.myrealtrip.com/offers?country=Netherlands&group_category=experience&order=reviews&page=2

### 페이지의 html 정보를 가져오는 함수

In [171]:
pd.DataFrame().append(pd.DataFrame([1]))

,0
0,1


In [172]:
def get_offer_info(urls):
    
    df = pd.DataFrame()
    try:
        for url in urls:
            html = urlopen(url)
            dom = BeautifulSoup(html, "html.parser")
            tmp_one = dom.find_all('script', 'js-react-on-rails-component')[1].get_text()
            info_dict = json.loads(tmp_one)
            offer_info = info_dict["offers"][:10] ### 첫번쨰 페이지에서 10개 상품만 가져오도록 설정
            df = df.append(get_result_pd(offer_info)) ### html 에서 json을 찾아내어 상품의 정보를 가져오는 함수로 보내고, DataFrame에 누적
    except:
        for url in urls:
            html = urlopen(url)
            dom = BeautifulSoup(html, "html.parser")
            tmp_one = dom.find_all('script', 'js-react-on-rails-component')[1].get_text()
            info_dict = json.loads(tmp_one)
            offer_info = info_dict["offers"] ### 페이지에 10개 미만의 여행 상품이 있을 경우 예외처리
            df = df.append(get_result_pd(offer_info))
        
    return df

### 페이지 내의 상품 코드와 간단한 정보를 가져오는 함수

In [173]:
def get_result_pd(offer_info):
    
    col_names = ["country","city_name", "code", "guide_type", "title"]
    
    dfs = []
    for x in offer_info:
        dfs.append({
            "country":x["country"]["key"],
            "city_name":x["city"]["key"],
            "code":x["id"],
            "guide_type":x["category"],
            "title":x["title"],
        })
        
    return pd.DataFrame(dfs, columns=col_names)

### get_url에 nation_names를 넣어서 실행하면, 
- 국가별 url을 가져오고,
- 국가별 url에서 후기순으로 정렬된 전체상품 페이지에서 html을 가져오고,
- 10개의 데이터에서 여행상품 정보를 가져와 DataFrame으로 출력해준다.

In [174]:
result = get_url(nation_names)
result.reset_index(drop=True, inplace=True)

In [175]:
result

,country,city_name,code,guide_type,title
0,France,Paris,53666,여행편의,"[공항유심] -더블데이터 프로모션 - KPN유심, TDC유심, 쓰리유심 ( 수령비 무료)"
1,Germany,Munchen,18727,여행편의,*반짝특가! 이게실화냐* 유럽 통합 40개국 플레이와이파이 포켓와이파이 + 보조배터리
2,Germany,Munchen,34163,여행편의,[플레이유심]데이터2배제공 이벤트중 KPN/쓰리/EU 유럽유심 (공항수령무료/출국전...
3,Germany,Dusseldorf,60636,여행편의,"[무료 배송] 유럽 유심 - EE/Three/KPN 유심 5종 / (영국 유심, 프..."
4,United Kingdom,London,66554,여행편의,"[인천공항 수령] 유럽 유심 - EE/Three 유심 3종 / (영국 유심, 프랑스..."
...,...,...,...,...,...
917,"Tanzania, United Republic of",Moshi,49568,가이드 투어,[투어] 탄자니아 야생동물 사파리 투어 (3일) :: 탄자니아/아루샤
918,"Tanzania, United Republic of",Moshi,49564,가이드 투어,[투어] 킬리만자로 트래킹 (3일) :: 탄자니아/모시
919,"Tanzania, United Republic of",Moshi,41643,가이드 투어,[투어] 세렝게티 야생동물 사파리 투어 (6일) :: 탄자니아/아루샤
920,"Tanzania, United Republic of",Moshi,41883,가이드 투어,[투어] 킬리만자로 트래킹/등반 (6일) :: 탄자니아/모시


# CSV 로 저장

In [181]:
result.to_csv("./nation_city_offer_code_list.csv")

In [180]:
result.loc[result["country"] == "United Kingdom"]

,country,city_name,code,guide_type,title
4,United Kingdom,London,66554,여행편의,"[인천공항 수령] 유럽 유심 - EE/Three 유심 3종 / (영국 유심, 프랑스..."
5,United Kingdom,London,28484,여행편의,[말톡] 유럽유심 보다폰 쓰리 KPN 오렌지 등 전체 통신사 공항수령무료
17,United Kingdom,London,66554,여행편의,"[인천공항 수령] 유럽 유심 - EE/Three 유심 3종 / (영국 유심, 프랑스..."
23,United Kingdom,London,66554,여행편의,"[인천공항 수령] 유럽 유심 - EE/Three 유심 3종 / (영국 유심, 프랑스..."
29,United Kingdom,London,76622,여행편의,"[유럽 유심] Vodafone 유럽 통합유심 데이터 3GB, 10GB, 20GB /30일"
44,United Kingdom,London,66554,여행편의,"[인천공항 수령] 유럽 유심 - EE/Three 유심 3종 / (영국 유심, 프랑스..."
57,United Kingdom,London,66554,여행편의,"[인천공항 수령] 유럽 유심 - EE/Three 유심 3종 / (영국 유심, 프랑스..."
63,United Kingdom,London,66554,여행편의,"[인천공항 수령] 유럽 유심 - EE/Three 유심 3종 / (영국 유심, 프랑스..."
70,United Kingdom,London,1829,가이드 투어,[후기1400개] 코츠월드+옥스포드 #점심제공 #빠른입장 #옥코1위
71,United Kingdom,London,315,가이드 투어,런던의 과거와 현재를 탐방하는 시내 워킹투어


# 4. 코드를 가져와서 세부 컨텐츠를 가져오는 함수
- 평점, 판매수, 상품소개, 리뷰갯수, 여행목적(company), 추천 연령대, 여행 추천 태그

### 나라별 10개씩 파싱한 상품 코드를 새로운 변수에 담아준다.

In [184]:
codes = result["code"]

In [185]:
codes

0      53666
1      18727
2      34163
3      60636
4      66554
       ...  
917    49568
918    49564
919    41643
920    41883
921    41903
Name: code, Length: 922, dtype: int64

In [186]:
def get_content(code):
    
    col_names = ["id","score", "selling_count", "introduction", "reviews_count", "travel_purpose", "travel_age", "tag_list"]
    
    df = []
    for idx in code:
        print("=",end="")
        url_2 = "https://www.myrealtrip.com/offers/{}".format(idx)
        html = urlopen(url_2)
        dom = BeautifulSoup(html, "html.parser")
        tmp_one = dom.find_all('script', 'js-react-on-rails-component')[1].get_text()
        content_dict = json.loads(tmp_one)
        content_info = content_dict["offerInfo"]
        
        df.append({
            "id":content_info["id"],
            "score":content_info["score"],
            "selling_count":content_info["selling_count"],
            "introduction":content_info["introduction"],
            "reviews_count":content_info["total_traveler_reviews"],
            "travel_purpose":content_info["preferred_travel_concept"],
            "tag_list":content_info["tag_list"],
        })

    return pd.DataFrame(df, columns=col_names)

### 코드 리스트의 코드를 가져와서 상품의 세부페이지 정보를 DataFrame으로 출력해준다.

In [188]:
result_content = get_content(codes)

In [189]:
result_content

,id,score,selling_count,introduction,reviews_count,travel_purpose,travel_age,tag_list
0,53666,4.409020,3179,[ 구매 시 꼭 확인하세요 ]\r\n\r\n1. 상품안내\r\n\r\n** 인천공...,643,"{'purpose': 'etc', 'age_group': '50'}",NaN,"[통역/비즈니스, 로컬투어]"
1,18727,4.123636,2546,"[인천 1, 2터미널 , 김해, 김포] 유럽 플레이와이파이 포켓와이파이 \r\n★ ...",550,"{'purpose': 'lover', 'age_group': '30'}",NaN,[로컬투어]
2,34163,4.372439,2401,서비스명 : 유럽 통합 유심 플레이유심 (공항수령 무료)\r\n\r\n** 국내 ...,537,"{'purpose': 'friends', 'age_group': '20'}",NaN,[]
3,60636,4.252083,3366,※ 유효기간 안내 ※\r\nKPN 유심 데이터 4GB(2020.08.31)\r\nK...,480,"{'purpose': 'honeymoon', 'age_group': '20'}",NaN,[]
4,66554,4.313107,2544,[인천공항 1T 수령] 유럽 유심 - EE/Three 유심 3종\r\n[인천공항 1...,412,"{'purpose': 'parents', 'age_group': '30'}",NaN,[]
...,...,...,...,...,...,...,...,...
917,49568,0.000000,0,아프리카를 체험할 수 있는 탄자니아 국립공원... \r\n\r\n끝이 보이지 않는 ...,0,None,NaN,"[국립공원, 캠핑]"
918,49564,0.000000,0,"아프리카를 체험할 수 있는 킬리만자로...\r\n\r\n아프리카 대륙의 최고봉(5,...",0,None,NaN,"[국립공원, 캠핑]"
919,41643,0.000000,0,아프리카를 체험할 수 있는 킬리만자로 세렝게티 국립공원\r\n\r\n끝이 보이지 않...,0,None,NaN,"[국립공원, 캠핑]"
920,41883,0.000000,0,아프리카를 체험할 수 있는 킬리만자로... 세렝게티...\r\n\r\n아프리카 대륙...,0,None,NaN,"[국립공원, 캠핑]"


# CSV 파일로 저장

In [190]:
result_content.to_csv("./code_content.csv", index=False)

In [191]:
result

,country,city_name,code,guide_type,title
0,France,Paris,53666,여행편의,"[공항유심] -더블데이터 프로모션 - KPN유심, TDC유심, 쓰리유심 ( 수령비 무료)"
1,Germany,Munchen,18727,여행편의,*반짝특가! 이게실화냐* 유럽 통합 40개국 플레이와이파이 포켓와이파이 + 보조배터리
2,Germany,Munchen,34163,여행편의,[플레이유심]데이터2배제공 이벤트중 KPN/쓰리/EU 유럽유심 (공항수령무료/출국전...
3,Germany,Dusseldorf,60636,여행편의,"[무료 배송] 유럽 유심 - EE/Three/KPN 유심 5종 / (영국 유심, 프..."
4,United Kingdom,London,66554,여행편의,"[인천공항 수령] 유럽 유심 - EE/Three 유심 3종 / (영국 유심, 프랑스..."
...,...,...,...,...,...
917,"Tanzania, United Republic of",Moshi,49568,가이드 투어,[투어] 탄자니아 야생동물 사파리 투어 (3일) :: 탄자니아/아루샤
918,"Tanzania, United Republic of",Moshi,49564,가이드 투어,[투어] 킬리만자로 트래킹 (3일) :: 탄자니아/모시
919,"Tanzania, United Republic of",Moshi,41643,가이드 투어,[투어] 세렝게티 야생동물 사파리 투어 (6일) :: 탄자니아/아루샤
920,"Tanzania, United Republic of",Moshi,41883,가이드 투어,[투어] 킬리만자로 트래킹/등반 (6일) :: 탄자니아/모시


# 5. 처음 파싱한 국가별 코드 데이터에서 코드가 중복인 것 정리

In [192]:
result = result.drop_duplicates('code', keep='first')

In [193]:
result

,country,city_name,code,guide_type,title
0,France,Paris,53666,여행편의,"[공항유심] -더블데이터 프로모션 - KPN유심, TDC유심, 쓰리유심 ( 수령비 무료)"
1,Germany,Munchen,18727,여행편의,*반짝특가! 이게실화냐* 유럽 통합 40개국 플레이와이파이 포켓와이파이 + 보조배터리
2,Germany,Munchen,34163,여행편의,[플레이유심]데이터2배제공 이벤트중 KPN/쓰리/EU 유럽유심 (공항수령무료/출국전...
3,Germany,Dusseldorf,60636,여행편의,"[무료 배송] 유럽 유심 - EE/Three/KPN 유심 5종 / (영국 유심, 프..."
4,United Kingdom,London,66554,여행편의,"[인천공항 수령] 유럽 유심 - EE/Three 유심 3종 / (영국 유심, 프랑스..."
...,...,...,...,...,...
917,"Tanzania, United Republic of",Moshi,49568,가이드 투어,[투어] 탄자니아 야생동물 사파리 투어 (3일) :: 탄자니아/아루샤
918,"Tanzania, United Republic of",Moshi,49564,가이드 투어,[투어] 킬리만자로 트래킹 (3일) :: 탄자니아/모시
919,"Tanzania, United Republic of",Moshi,41643,가이드 투어,[투어] 세렝게티 야생동물 사파리 투어 (6일) :: 탄자니아/아루샤
920,"Tanzania, United Republic of",Moshi,41883,가이드 투어,[투어] 킬리만자로 트래킹/등반 (6일) :: 탄자니아/모시


# 6. 상품 세부 데이터에서 코드가 중복인 것 정리

In [196]:
result_content = result_content.drop_duplicates('id', keep='first')

In [197]:
result_content

,id,score,selling_count,introduction,reviews_count,travel_purpose,travel_age,tag_list
0,53666,4.409020,3179,[ 구매 시 꼭 확인하세요 ]\r\n\r\n1. 상품안내\r\n\r\n** 인천공...,643,"{'purpose': 'etc', 'age_group': '50'}",NaN,"[통역/비즈니스, 로컬투어]"
1,18727,4.123636,2546,"[인천 1, 2터미널 , 김해, 김포] 유럽 플레이와이파이 포켓와이파이 \r\n★ ...",550,"{'purpose': 'lover', 'age_group': '30'}",NaN,[로컬투어]
2,34163,4.372439,2401,서비스명 : 유럽 통합 유심 플레이유심 (공항수령 무료)\r\n\r\n** 국내 ...,537,"{'purpose': 'friends', 'age_group': '20'}",NaN,[]
3,60636,4.252083,3366,※ 유효기간 안내 ※\r\nKPN 유심 데이터 4GB(2020.08.31)\r\nK...,480,"{'purpose': 'honeymoon', 'age_group': '20'}",NaN,[]
4,66554,4.313107,2544,[인천공항 1T 수령] 유럽 유심 - EE/Three 유심 3종\r\n[인천공항 1...,412,"{'purpose': 'parents', 'age_group': '30'}",NaN,[]
...,...,...,...,...,...,...,...,...
917,49568,0.000000,0,아프리카를 체험할 수 있는 탄자니아 국립공원... \r\n\r\n끝이 보이지 않는 ...,0,None,NaN,"[국립공원, 캠핑]"
918,49564,0.000000,0,"아프리카를 체험할 수 있는 킬리만자로...\r\n\r\n아프리카 대륙의 최고봉(5,...",0,None,NaN,"[국립공원, 캠핑]"
919,41643,0.000000,0,아프리카를 체험할 수 있는 킬리만자로 세렝게티 국립공원\r\n\r\n끝이 보이지 않...,0,None,NaN,"[국립공원, 캠핑]"
920,41883,0.000000,0,아프리카를 체험할 수 있는 킬리만자로... 세렝게티...\r\n\r\n아프리카 대륙...,0,None,NaN,"[국립공원, 캠핑]"


In [198]:
result_content.iloc[0, 3]

'[ 구매 시 꼭 확인하세요 ]\r\n\r\n1. 상품안내\r\n\r\n**  인천공항 씨티,sm면세점 - 현금처럼 사용가능한 1만원($30이상 구매시 사용가능) 쿠폰 제공됩니다. - 심카드와 같이 드려요!\r\n**  주류, 담배 포함입니다.  - 담배는 2보루 구매시 1만원 할인되세요 !!\r\n\r\n1 - 1. 유럽유심 터키 + 36개국 1GB 15일 \r\n      -  터키가 되는 유럽 유심\r\n      -  데이터 전용 / 통화 문자 불가\r\n      -  유럽일정이 짧은 분, 저렴한 유럽 유심 찾는 분께 추천 드려요\r\n\r\n* 유럽유심 터키+36개국 사용가능 국가 \r\n터키, 영국, 독일, 이탈리아, 네덜란드, 스위스, 스페인, 프랑스, 모나코, 벨기에, 오스트리아, 덴마크, \r\n노르웨이, 스웨덴, 아이슬란드, 핀란드, 몰타, 룩셈부르크, 아일랜드, 그리스, 포르투갈, 폴란드, \r\n체코, 헝가리, 루마니아, 불가리아, 키프로스, 에스토니아, 라트비아, 리투아니아, 크로아티아, \r\n슬로바키아, 슬로베니아, 리히텐슈타인\r\n(주의!)  - 러시아, 보스니아, 세르비아, 모로코는 사용 불가입니다.\r\n\r\n1 - 2. KPN 유심   -  데이터 프로모션 진행중\r\n      - 전 유럽 4G LTE 속도 + 유럽 통화 60분  [우버 가능해요]\r\n      -  유럽 내 가장 빠른 속도 + 유럽통화 60분  - 고퀄리티 우수한  품질!! \r\n\r\n   * 더블 데이터 프로모션  진행중 4GB -> 6GB / 8GB -> 12GB  사용 가능해요! \r\n   * 프로모션 추가 데이터 사용을 원하시면 한국에서 미리 위챗앱을 가입해주세요.\r\n      (한국에서 미리 위챗앱 가입하셔야 하며 가입 불가의 경우 저희가 도움 드릴 수 없습니다. 주문 전 미리 확인해주세요)\r\n   * 방법은 공항유심센터 홈페이지 유럽에서 KPN 유심 페이지에 안내되어 있습니다! \r\n   *  kpn 더블데이터 프로모션은 반

# 7. 상품 리뷰 가져오는 함수
- html 구조상 beautifulsoup 기능으로는 ㅡ리뷰 데이터의 "더보기" 버튼 누르기전 첫 3개만 가져올 수 있다. 

In [4]:
!ls ./

chromedriver.exe
code_content.csv
content_0_25.csv
Crawling.ipynb
crawling_reviews.ipynb
mrt_�굹�씪蹂� 肄붾뱶 異붽��濡� �겕濡ㅻ쭅.ipynb
nation_city_offer_code_list.csv
Untitled.ipynb


### code 데이터 불러오기

In [13]:
code = pd.read_csv("nation_city_offer_code_list.csv")
del code["Unnamed: 0"]

In [14]:
code_list = code["code"]
code_list = pd.DataFrame(code_list)
code_list

,code
0,53666
1,18727
2,34163
3,60636
4,66554
...,...
917,49568
918,49564
919,41643
920,41883


In [15]:
code_list["count"] = 1
code_list

,code,count
0,53666,1
1,18727,1
2,34163,1
3,60636,1
4,66554,1
...,...,...
917,49568,1
918,49564,1
919,41643,1
920,41883,1


In [16]:
code_groupby = code_list.groupby("code").sum()
code_groupby = code_groupby.sort_values(by="count", ascending=False)

In [17]:
code_groupby

,count
code,
53666,27
34163,26
35342,21
66554,15
60636,15
...,...
27491,1
27527,1
27564,1


### 코드 리스트에서 중복된 코드 제거

In [18]:
code_final = code_list.drop_duplicates('code', keep='first')
code_final.reset_index(drop=True, inplace=True)

In [19]:
code_final["code"]

0      53666
1      18727
2      34163
3      60636
4      66554
       ...  
733    49568
734    49564
735    41643
736    41883
737    41903
Name: code, Length: 738, dtype: int64

In [31]:
codes = code_final["code"][0:]
codes

0      53666
1      18727
2      34163
3      60636
4      66554
       ...  
733    49568
734    49564
735    41643
736    41883
737    41903
Name: code, Length: 738, dtype: int64

In [32]:
import pandas as pd
import requests
import re

from scrapy.http import TextResponse

In [33]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

### 상품 페이지로 이동하기 위해 코드가 포함 된 url을 만들고 이동하여 html 데이터를 가져오는 함수

In [34]:
def get_review(codes):
    
    df = pd.DataFrame()
    for idx in codes:  
        print(idx, end=" ")
        url = "https://www.myrealtrip.com/offers/{}".format(idx)
        html = urlopen(url)
        dom = BeautifulSoup(html, "html.parser")
        tmp_one = dom.find_all('script', 'js-react-on-rails-component')[1].get_text()
        content_dict = json.loads(tmp_one)
        review_info = content_dict["reviews"]
        df = df.append(get_result_pd(review_info, idx))
    
    return df

### html 데이터에서 상품 코드와 리뷰랄 가져와 DataFrame 으로 출력하는 함수

In [35]:
def get_result_pd(review_info, idx):
    col_names = ["code", "review_1"]
    datas = []
    for x in review_info:
        datas.append({
            "code":idx,
            "review_1":x["message"].replace("\n", ""),
        })

    return pd.DataFrame(datas, columns=col_names)

### 738개 코드의 리뷰 3개씩 가져오도록 실행

In [36]:
result_1 = get_review(codes)

53666 18727 34163 60636 66554 28484 2122 1474 34700 460 8693 12350 3108 1899 29102 53676 35342 7782 30767 34186 76622 50891 50896 50894 38847 13724 10423 81298 81296 77139 5929 19954 3554 18032 5097 35776 62071 29382 26442 8768 14457 14318 26780 10116 46016 14235 1829 315 11138 1867 2246 40905 27564 27527 48505 1562 67040 18661 2007 2398 7511 470 4725 5638 2114 7212 32691 6349 2654 5047 7366 10322 48868 2821 53690 32624 37376 61875 41487 18962 30028 30027 9054 13919 22345 77052 77051 77050 77047 77046 77040 77039 77049 47381 45020 58667 34416 73602 73284 62035 58646 37577 51399 25575 52976 34439 34438 34437 34436 34434 79729 62653 2509 27814 27915 34188 11407 8657 6559 7608 1334 6312 16588 7220 11701 15057 38211 5087 34867 34868 77818 77817 77816 77815 77814 77824 77811 76722 43691 42665 14922 51215 7787 55229 7795 3608 62526 36982 47103 45262 32452 48672 36308 33556 53192 14231 58965 78497 78494 78486 78483 78481 78475 65811 55623 52325 46801 55609 8490 1476 1297 37726 16081 9696 4248

### 코드에 해당하는 리뷰데이터가 3개의 dict 로 나누어져 있어서, 한 코드의 3개의 행이 만들어 진다.

In [37]:
result_1

,code,review_1
0,53666,좋긴 한데 어느 유심이나 그렇듯 3G 전환이 잦습니다
1,53666,잘 터져요!!
2,53666,굿!!!!!!!!!
0,18727,유럽은 원래 와이파이가 좀 느릴 수 있다고 알고 있었는데 우리나라보단 확실히 느리지...
1,18727,파리에서 사용했는데 전날 충전해 놓고 아침부터 하루종일 잘 사용했어요 :)
...,...,...
0,27627,좋아요
1,27627,생각보다 너무 오래타야해서 손이 아팠어요ㅠ 그리고 배두인 마을이 아니라 그냥 배두인...
0,46232,친절한 가이드와 자세한 역사적인 설명을 들을 수 있었던 시간이었습니다. 추천드립니다.
0,46067,"예약자 두명과 운전기사 한명, 가이드 한명 총네명이 승용차에 타고 피라미드를 보러가..."


### 3개의 행으로 나누어진 리뷰를 하나의 코드로 합치는 코드

In [38]:
data = []
for z in codes:
    data.append({"code":z,
                 "review":result_1.loc[result_1["code"] == z, "review_1"].sum()})

In [40]:
code_review = pd.DataFrame(data, columns=["code", "review"])
code_review

,code,review
0,53666,좋긴 한데 어느 유심이나 그렇듯 3G 전환이 잦습니다잘 터져요!!굿!!!!!!!!!
1,18727,유럽은 원래 와이파이가 좀 느릴 수 있다고 알고 있었는데 우리나라보단 확실히 느리지...
2,34163,kpn으로 5+5기가로 구매했는데 런던/동유럽에서는 잘터졌지만 스페인에서는 조금 느...
3,60636,처음에 유심 끼우고 잡는데 시간이 걸리긴 하는데 그 이후로는 문제없이 잘쓰고 다녔습...
4,66554,"연결이 잘됩니다.데이터 용량이 큽니다.KPN 유심칩을 사용하였습니다. 지하철, 건물..."
...,...,...
733,49568,0
734,49564,0
735,41643,0
736,41883,0


# CSV 파일로 저장

In [41]:
code_review.to_csv("./code_review.csv", index=False)